# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Milvus

In [27]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

In [28]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

[Document(metadata={'pk': 460219909630597135, 'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 13}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [29]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://129.201.70.35:11434', model='qwen3:8B', temperature=0.1)

my_llm.invoke("你好")

'<think>\n嗯，用户发来“你好”，我需要回应。首先，要友好，用中文。然后，可能用户想开始对话，所以应该主动询问他们有什么需要帮助的。保持自然，不要太机械。比如可以说“你好！有什么我可以帮你的吗？”或者更亲切一点，比如“你好呀！今天过得怎么样？”不过用户可能只是测试，所以保持简洁和开放。另外，注意不要用太复杂的句子，保持口语化。可能用户之后会问问题，所以准备好回答各种问题。总之，回应要友好、开放，鼓励用户继续对话。\n</think>\n\n你好！有什么我可以帮你的吗？😊'

In [ ]:
# from langchain_community.llms import VLLMOpenAI  # 注意类名为 VLLMOpenAI[3](@ref)
# my_llm = VLLMOpenAI(
#     openai_api_key="token-abc123",          # vLLM 无需鉴权，设为空字符串[3](@ref)
#     openai_api_base="http://129.201.70.35:9991/v1",  # 服务端地址
#     model_name="my_qwen3_8b",  # 需与部署的模型路径一致
#     max_tokens=1024,                # 控制生成文本最大长度
#     temperature=0,               # 生成多样性参数（0~1）
#     top_p=0.9,                      # 采样阈值
#     streaming=True                  # 支持流式输出（可选）
# )
# response = my_llm.invoke("你是谁？/no_think")
# response

## 3. 构建检索问答链

prompts

In [30]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question", "sources"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [31]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [34]:
question_1 = "什么是ZXVMAX-S的5GC业务？/no_think"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
# print(result["source_documents"])

大模型+知识库后回答 question_1 的结果：
<think>

</think>

ZXVMAX-S的5GC业务是指ZXVMAX-S多维价值分析系统在5G核心网（5G Core Network, 5GC）场景中的应用。该系统面向用户的网络运维和运营分析，能够从网元、用户、终端、业务等多个维度，对业务使用过程中的质量和特征进行全方位的挖掘。它支持实时分析（如网络质量、数据业务质量等）和事后分析（如用户投诉、问题分析等），为移动通信网络的运维和运营提供全面支撑。在5GC业务中，ZXVMAX-S系统能够帮助运营商优化网络性能、提升服务质量，并为市场发展提供数据支撑。谢谢你的提问！


In [38]:
question_2 = "严威是谁？/no_think"
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
# print(result["result"])

大模型+知识库后回答 question_2 的结果：


### 4.2 无知识库大模型自己回答的效果

In [9]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

/tmp/ipykernel_1270322/46941173.py:5: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  my_llm.predict(prompt_template)


'<think>\n好的，我现在要回答用户的问题：“什么是vmax的5GC业务？”首先，我需要明确用户提到的“vmax”指的是什么。因为“vmax”可能是一个公司、产品或技术的名称，但我不太确定。首先，我应该检查是否有常见的公司或技术使用“vmax”作为缩写。\n\n首先，我想到的是，可能用户指的是“vMAX”这个产品，比如由Dell EMC公司推出的vMAX存储系统。不过，vMAX通常与存储解决方案相关，而不是5GC业务。5GC（5G Core）是5G网络的核心网部分，涉及网络功能虚拟化（NFV）和云原生架构。因此，如果用户提到的是vMAX的5GC业务，可能是指某个公司利用vMAX技术或平台来支持5GC服务。\n\n接下来，我需要确认是否存在名为“vMAX”的5GC相关产品或服务。可能用户将“vMAX”与5GC结合，指的是某个特定的解决方案。例如，某些公司可能将他们的存储或虚拟化平台用于5GC的部署，但这种情况比较少见。\n\n另外，也有可能用户存在拼写错误，比如“vMAX”是否应为其他名称，如“vMAG”或其他缩写。但根据现有信息，我需要假设用户确实指的是“vMAX”。\n\n接下来，我需要考虑5GC业务的基本概念。5GC是5G核心网的一部分，负责处理用户数据、移动性管理、会话管理等。它基于NFV和云原生技术，支持灵活的网络切片和边缘计算。因此，如果vMAX与5GC相关，可能涉及虚拟化、云原生架构、网络切片管理等方面。\n\n假设vMAX是一个虚拟化平台或云原生解决方案，那么vMAX的5GC业务可能指的是利用该平台来部署和管理5G核心网功能。例如，vMAX可能提供虚拟化网络功能（VNF）的部署、资源管理、自动化运维等服务，支持运营商构建灵活、可扩展的5GC网络。\n\n不过，如果用户提到的vMAX并非指存储产品，而是另一个公司或技术，可能需要更多的上下文。例如，某些公司可能有自己的5GC解决方案，命名为vMAX。但目前没有广泛认知的vMAX与5GC直接关联的案例。\n\n因此，可能的结论是，用户可能混淆了术语，或者vMAX指的是某个特定公司的产品，该产品支持5GC业务。在这种情况下，我需要解释5GC业务的一般概念，并指出vMAX可能涉及的方面，同时建议用户确认具体上下文。\n\n总结来说，我需要先澄清vMAX的含义，然后结合5GC业务的定义，解释可能的关联。如果

In [10]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

'<think>\n好的，用户问“严威是谁？”，我需要先确定这个“严威”指的是谁。首先，可能是一个公众人物，比如演员、歌手、作家，或者某个领域的专家。但“严威”这个名字比较常见，可能有多个同名的人。\n\n首先，我应该考虑是否有名人叫严威。比如，演员方面，有没有叫严威的？我记得可能有演员严宽，但不确定有没有严威。或者可能是指其他领域的，比如作家、科学家、企业家等。\n\n另外，用户可能是在问某个特定领域的人物，比如科技、文化、体育等。如果用户没有提供更多背景，可能需要给出多个可能的解释。\n\n也有可能用户指的是某个虚构角色，比如小说或影视作品中的角色，但这种情况可能性较低，除非有特别的上下文。\n\n接下来，我需要检查是否有知名人物叫严威。比如，搜索一下，发现严威可能是一个演员，但不确定。或者可能是指某个历史人物？比如古代的某个官员或学者？\n\n另外，也有可能用户打错了名字，比如“严威”是否是“严宽”或其他名字的误写？这时候可能需要提示用户确认名字的正确性。\n\n如果无法确定具体是谁，应该说明可能有多个同名人物，并列举一些可能的选项，同时建议用户提供更多信息以便更准确的回答。\n\n总结一下，可能的步骤是：1. 确认名字的正确性；2. 检查是否有知名人物；3. 提供可能的解释；4. 建议用户提供更多背景信息。\n</think>\n\n“严威”是一个较为常见的中文名字，可能指代不同领域的人物，具体身份需结合上下文判断。以下是几种可能的解释：\n\n1. **演员/艺人**  \n   中国内地演员**严宽**（本名严威）曾因出演《仙剑奇侠传三》中的李逍遥而广为人知。他于2005年因该剧走红，后逐渐淡出娱乐圈。\n\n2. **历史人物**  \n   古代可能有同名人物，但需更多背景信息确认。\n\n3. **其他领域**  \n   可能是作家、学者、企业家等，但需具体信息进一步核实。\n\n**建议**：若需更准确的回答，请补充更多背景（如领域、相关事件等）。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

/tmp/ipykernel_1270322/2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [13]:
questions = [
    "什么是ZXVMAX-S的5GC业务？",
    "包含哪些功能", 
    "整理成表格"  
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

回答：<think>
好的，用户问的是“什么是ZXVMAX-S的5GC业务？”，我需要根据提供的上下文来回答。首先，我得仔细看看相关上下文里有没有提到ZXVMAX-S的5GC业务相关内容。

在上下文中，有提到“ZXVMAX-S多维价值分析系统产品描述（5GC业务）图5-8会话跟踪组网逻辑图”，这说明ZXVMAX-S的5GC业务可能与产品描述和组网逻辑有关。另外，其他部分主要涉及告警处理、接口完整率和合规率的问题，可能和5GC业务的具体功能或应用场景相关。

不过，上下文中的信息比较零散，没有直接定义5GC业务是什么。可能需要结合已知的5GC知识来补充。5GC（5G Core）是5G网络的核心部分，负责控制面和用户面功能，支持网络切片、边缘计算等特性。ZXVMAX-S作为多维价值分析系统，可能在5GC业务中用于数据收集、分析，帮助运营商优化网络性能和业务策略。

但根据用户提供的上下文，重点可能在于ZXVMAX-S系统如何支持5GC业务，比如通过会话跟踪、接口监控等。因此，回答应结合产品描述和组网逻辑，说明ZXVMAX-S在5GC中的角色，如数据分析、性能监控等，同时提到相关告警处理机制确保服务质量和合规性。

需要注意的是，用户提供的上下文可能没有详细说明5GC业务的具体定义，所以回答需要基于现有信息，并适当扩展，但避免超出上下文范围。最后，确保回答结束时有“谢谢你的提问！”。
</think>

ZXVMAX-S的5GC业务是指基于5G核心网（5G Core）的多维价值分析系统，主要用于支持5G网络中的业务数据分析、性能监控及优化。该系统通过采集和分析5GC（5G Core Cloud）相关接口（如S11、SGS等）的数据，帮助运营商实现对网络切片、会话管理、服务指标等关键业务的精细化运营，从而提升网络效率和用户体验。其核心功能包括告警处理、填充率监控（如接口完整率、合规率）以及组网逻辑分析（如会话跟踪），确保5GC业务的稳定运行和资源优化。  

谢谢你的提问！


回答：<think>
好的，用户现在问的是ZXVMAX-S的5GC业务包含哪些具体功能。我需要根据之前的对话历史和上下文来回答。首先，回顾之前的对话，用户已经询问过ZXVMAX-S的5GC业务是什么，而我的回答提到了多维价值分析系统，涉及5GC的组网逻辑、告警处理、接口完整率和合规率监控等。

现